In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp '/content/drive/MyDrive/Module 3 updated/Module 3' -r '/content/'


In [ ]:
!pip install selfcheckgpt

In [ ]:
NGROK_TOKEN = "2fUPdJfd9Mh3y7aRrmzlbg7aKvU_4rrEDg62BACmd5gn5bZ85"

!pip install flask
!pip install pyngrok

from pyngrok import ngrok, conf
ngrok.set_auth_token(NGROK_TOKEN)

In [ ]:

import torch
from selfcheckgpt.modeling_mqag import MQAG

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

mqag_model = MQAG(
    g1_model_type='race', # race (more abstractive), squad (more extractive)
    device=device
)

cuda
MQAG (race) initialized to cuda


In [ ]:


%cd '/content/Module 3/'

/content/Module 3


In [ ]:
import numpy as np

def generate_questions_answers(context):
  questions = mqag_model.generate(context=context, do_sample=True, num_questions=10)
  final_set = []
  for i, question_item in enumerate(questions):
      question = question_item['question']
      options = question_item['options']

      single_question = [{'question': question, 'options': options}]
      probs = mqag_model.answer(questions=single_question, context=context)
      correct_answer = options[np.argmax(probs)]

      final_set.append({'question': question, 'options': options, 'answer': correct_answer})

  return final_set

In [ ]:


from flask import Flask, render_template, request, redirect, url_for , jsonify
import json
from pyngrok import ngrok



app = Flask(__name__)


global answers
global quiz_data_v2
global questionText
questionText = ""

previous_scores={}



with open("details.json", "r") as jf:
    data = json.load(jf)
    users = data["login"]

previous_scores=data['scores']
#previous_scores = data['scores']

@app.route('/', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        # Retrieve form data
        email = request.form.get('login-email')
        password = request.form.get('login-password')

        if email in users.keys() and users[email]==password:
            return redirect(url_for('generate_qna'))

    return render_template('auth.html')


@app.route('/signup', methods=['GET', 'POST'])
def signup():
    if request.method == 'POST':
        # Retrieve form data
        email = request.form.get('signup-email')
        password = request.form.get('signup-password')
        password2 = request.form.get('confirm-signup-password')

        # Perform validation or other actions with the form data
        with open("details.json", "w") as jfk:
            users[email] = password
            json.dump(users, jfk)

        # Print form data (for demonstration purposes)
        print(f'Email: {email}, Password: {password} ,  password2 : { password2}')

        # Add your logic for user registration or other actions
        return redirect(url_for('login'))

    return render_template('auth.html')




@app.route('/create_post' , methods=['GET', 'POST'] )
def create_post():
    # Additional logic for generating Q&A or render a template
     if request.method == 'POST':
            # Retrieve form data
        new_post = request.form.get('post-input')


        # Perform validation or other actions with the form data

        # Print form data (for demonstration purposes)
        print(f'new_post: {new_post}')
        return redirect(url_for('forum'))


@app.route('/subject_name', methods=['POST'])
def subject_name():
  global subjectName
  data = request.json
  subjectName = data.get('subject')
  print("subjectName: ", subjectName)

  return jsonify({'message': 'Modal data received successfully'})


@app.route('/generate_qna' , methods=['GET', 'POST'])
def generate_qna():
    # Additional logic for generating Q&A or render a template
    question_data = []
    subjects = data['scores'].keys()
    for subj in subjects:
      question_data.append({"subject": subj, "num_questions": 10})

    return render_template('generate_qna.html' , questions=question_data)


@app.route('/answer_evaluation')
def answer_evaluation():

    # Retrieve data from the JSON file
    with open("details.json", "r") as jf:
        score_data = json.load(jf)

    subj_data = score_data["scores"]
    subjects_data = []
    for key, value in subj_data.items():
        subjects_data.append({"subject": key, "score": value})

    # subjects_data = [
    #     {"subject": list(subj_data.keys())[0], "score": subj_data[list(subj_data.keys())[0]]},
    #     {"subject": list(subj_data.keys())[1], "score": subj_data[list(subj_data.keys())[1]]},
    #     {"subject": list(subj_data.keys())[2], "score": subj_data[list(subj_data.keys())[2]]},
    # ]

    prev_values = list(previous_scores.values())
    print(prev_values)
    print(subjects_data)
    zipped_data = zip(subjects_data, prev_values)

    return render_template('answer_evaluation.html', zipped_data=zipped_data )
    # Additional logic for generating Q&A or render a template



@app.route('/forum')
def forum():
    # Additional logic for generating Q&A or render a template
    post_data = [
        {
            'title' : "Golanginya",
            'content'  : "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Consequat aliquet maecenas ut sit nulla"
        },
        {
            'title' : "Golanginya",
            'content'  : "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Consequat aliquet maecenas ut sit nulla"
        },
        {
            'title' : "Golanginya",
            'content'  : "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Consequat aliquet maecenas ut sit nulla"
        }
    ]
    return render_template('forum.html' ,post_data=post_data )


@app.route('/quiz', methods=['GET', 'POST'])
def quiz():
    global answers
    global quiz_data_v2
    global questionText
    quiz_data = [

    {
        "question": "For Fiscal year 2021, what was your total GhG Carbon Emission for all scopes?",
        "options": [
            "Scope 1 - Determine environmental impact levels.",
            "Scope 2 - Reduce carbon footprints.",
            "Scope 3 - Enhance environmental impacts on a larger scale.",
            "I do not know the answer to this question.",
        ],
    },
    {
        "question": "Another question?",
        "options": [
            "Option 1",
            "Option 2",
            "Option 3",
            "I do not know the answer to this question.",
        ],
    },


    # Add more questions and answers as needed
    ]
    if request.method == 'POST':
        print("post method ************")
        questionText = request.form.get('questionText')
        print(f"Launching quiz for subject: {questionText}")
        quiz_data = generate_questions_answers(questionText.replace('\n', ' '))

        quiz_data_v2 = []
        answers = []
        for item in quiz_data:
            # d = ast.literal_eval(item)
            js_data = {
                "question": item['question'],
                "answers": item['options']
            }
            quiz_data_v2.append(js_data)
            answers.append(item['answer'])
        print(quiz_data_v2)
        print("Answers:", answers)

        # return redirect(url_for('quiz'))
        return render_template('quiz.html', quiz_data=quiz_data_v2)

    return render_template('quiz.html', quiz_data=quiz_data_v2)


@app.route('/quiz/response', methods=['POST'])
def quiz_response():
    global quiz_data_v2
    global answers
    global previous_scores

    if request.method == 'POST':
        # Retrieve the selected answers from the form
        selected_answers = []

        for question in quiz_data_v2:
            selected_answers.append(request.form.get(f'selected-answer-{question["question"]}'))


        # You can now process the selected answers as needed
        # For example, you might want to store them in a database or perform some analysis

        # For now, let's just print the selected answers
        print("Selected Answers:", selected_answers)
        score = 0
        quiz_data_ans = []

        for i in range(len(selected_answers)):
          quiz_data_ans.append({'question': quiz_data_v2[i]['question'], 'answer':answers[i]})
          if selected_answers[i]==answers[i]:
            score += 10

        previous_scores=data['scores'].copy()
        print("prev" , previous_scores)
        data["scores"][subjectName] = score
        with open('details.json', 'w') as wf:
          json.dump(data, wf)

        result = "Fail" if score<50 else "Pass"


        # quiz_data_ans = [

        #         {
        #             "question": "For Fiscal year 2021, what was your total GhG Carbon Emission for all scopes?",
        #         "answer": "Scope 3 - Enhance environmental impacts on a larger scale."
        #     },
        #     {
        #         "question": "Another question?",
        #         "answer": "Option 3"
        #     },

        # ]

        # You can redirect to a different page or render a new template
        return render_template('QuizResult.html', result=result , score=score , quiz_data_ans=quiz_data_ans , que=questionText)
        # You might want to render a response or redirect to another page
    return render_template('quiz.html', quiz_data=quiz_data_v2 ,que=questionText )


@app.route('/logout')
def logout():
    # Additional logic for generating Q&A or render a template
    return redirect(url_for('login'))


ngrok_tunnel = ngrok.connect(5000)
print(ngrok_tunnel)
# print('Running at:', ngrok_tunnel.public_url)

if __name__ == '__main__':
    app.run(port=5000)


NgrokTunnel: "https://92f6-104-196-154-169.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [06/May/2024 16:30:40] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2024 16:30:41] "GET /static/js/auth.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2024 16:30:41] "GET /static/css/auth.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2024 16:30:42] "GET /static/Assets/Container.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2024 16:30:44] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/May/2024 16:30:54] "POST / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [06/May/2024 16:30:55] "GET /generate_qna HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2024 16:30:55] "GET /static/css/generate_qna.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2024 16:31:01] "POST /subject_name HTTP/1.1" 200 -


subjectName:  Math
post method ************
Launching quiz for subject: Algebra is the branch of mathematics that studies algebraic structures and the manipulation of statements within those structures. It is a generalization of arithmetic that introduces variables and algebraic operations other than the standard arithmetic operations such as addition and multiplication.
Initialized Generation


Input ids are automatically padded from 80 to 512 to be a multiple of `config.attention_window`: 512


Initialized Answering


...

Input ids are automatically padded from 81 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 74 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 67 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 75 to 512 to be a multiple of `config.attention_window`: 512
INFO:werkzeug:127.0.0.1 - - [06/May/2024 16:33:53] "POST /quiz HTTP/1.1" 200 -


[{'question': 'Algebra is the generalization of arithmetic with its specialty in _.', 'answers': ['adding and multiplication', 'manipulating statements in structure', 'variables and structures', 'manipulating statements']}, {'question': 'Algebra is a branch of mathematics that _.', 'answers': ['studies algebraic structures and manipulation of statements within those structures', 'studies number and its operation', 'introduces algebraic operations other than the standard arithmetic operations like addition and multiplication', 'introduces variables and algebraic operations other than the standard arithmetic operations such as addition and multiplication']}, {'question': 'Algebra is _.', 'answers': ['a branch of mathematics that studies algebraic structures and the manipulation of statements within them', 'a branch of mathematics that studies standard arithmetic operations', 'a generalization of arithmetic that introduces variables and algebraic operations', 'the branch of math that stud

INFO:werkzeug:127.0.0.1 - - [06/May/2024 16:33:54] "GET /static/css/quiz.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [06/May/2024 16:34:00] "GET /answer_evaluation HTTP/1.1" 200 -


[80, 90, 89, 92]
[{'subject': 'Math', 'score': 30}, {'subject': 'Science', 'score': 90}, {'subject': 'Python', 'score': 89}, {'subject': 'Networks', 'score': 92}]


INFO:werkzeug:127.0.0.1 - - [06/May/2024 16:34:01] "GET /static/css/answer_evaluation.css HTTP/1.1" 200 -
